In [9]:
# Your Fresh Direct login credentials
USERNAME = 'ilovegroceries@gmail.com'
PASSWORD = 'correcthorsebatterystaple'

# Your MessageBird api key
MESSAGE_BIRD_API_KEY = 'xxxxxxxxxxxxxxxxxxxxxxxxx'

# Phone number to text on grocery availability
PHONE_NUMBER_TO_TEXT = '+12125555555'

In [10]:
import os
import sys
import time
import json
import urllib
import datetime
import traceback

from lxml import html

try:
    import requests
except ImportError:
    # Install requests library to jupyter notebook env
    !{sys.executable} -m pip install requests
    
try:
    import messagebird
except ImportError:
    !{sys.executable} -m pip install messagebird
    
from requests import Request, Session
    
import logging

def setup_custom_logger(name):
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s',
                                  datefmt='%Y-%m-%d %H:%M:%S')
    handler = logging.FileHandler('log.txt', mode='w')
    handler.setFormatter(formatter)
    screen_handler = logging.StreamHandler(stream=sys.stdout)
    screen_handler.setFormatter(formatter)
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)
    logger.addHandler(handler)
    logger.addHandler(screen_handler)
    return logger

try:
    logger
except NameError:
    logger = setup_custom_logger('fresh-direct-poller')

In [11]:
class Alerter:
    def __init__(self, alert_interval=60):
        self.alert_interval = alert_interval
        self.last_alerted = datetime.datetime.fromtimestamp(0)
        
    def alert(self, message):
        if (datetime.datetime.now() - self.last_alerted).seconds > self.alert_interval:
            self.last_alerted = datetime.datetime.now()
            self.user_alert(message)
        
    def user_alert(self, message):
        pass  # Implement in subclass.
    
    
class TextAlerter(Alerter):
    client = messagebird.Client(MESSAGE_BIRD_API_KEY)
    def user_alert(self, message):
        message = self.client.message_create(
            'MessageBird',
            PHONE_NUMBER_TO_TEXT,
            message,
            {'reference': 'Foobar'}
        )

        
class FreshDirectClient:
    def __init__(self, logger):
        self.logger = logger
        self.auth_endpoint = 'https://www.freshdirect.com/api/login/'
        self.slots_endpoint = 'https://www.freshdirect.com/your_account/delivery_info_avail_slots.jsp'
        self.headers = {
            'User-Agent': 'PostmanRuntime/7.24.1',
            'Accept': '*/*',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive'
        }
        self.session = Session()

        
    def authenticate(self, user_id, password):        
        # Need to query homepage first to get session cookies for for auth.
        res1 = self.session.get('https://www.freshdirect.com/', timeout=(3, 5), headers=self.headers)
        res1.raise_for_status()
        
        #auth_json = '{{"userId":"{0}","password":"{1}"}}'.format(user_id, password)
        credentials = json.dumps({'userId': user_id, 'password': password})
        res2 = self.session.post(self.auth_endpoint, 
                                timeout=(3, 5), headers=self.headers, data={'data': credentials})
        res2.raise_for_status()
        
        self.logger.info("Authentication successful!")
        #_ = self.get_delivery_timeslots_html()


    def get_delivery_timeslots_html(self):
        res = self.session.get(self.slots_endpoint, headers=self.headers, timeout=10)
        self.logger.info(res.status_code)
        res.raise_for_status()
            
        return res.text
    
    
def parse_timeslots(html_string):
    tree = html.fromstring(html_string)

    available_slots = []
    for col in range(7):
        day_name = tree.xpath('//*[@id="ts_d{col}_hE_content"]/div[1]/b'.format(col=col))[0].text
        mmm, dd = tree.xpath('//*[@id="ts_d{col}_hE_content"]/div[2]'.format(col=col))[0].text.split(' ')
        logger.info('{0} {1} {2}'.format(day_name, mmm, dd))

        for row in range(7):
            time_slot = tree.xpath('//*[@id="ts_d{col}_ts{row}_time"]'.format(col=col, row=row))
            if not time_slot:
                continue

            time_slot_text = time_slot[0].text
            sold_out = 'tsSoldoutC' in list(time_slot[0].classes)

            if not sold_out:
                message = '{0} {1} {2} @ {3} {4}'.format(day_name, mmm, dd, time_slot_text, 'AVAILABLE')
                available_slots.append(message)

            logger.info('\t{} {}'.format(time_slot_text, 'SOLD OUT' if sold_out else 'AVAILABLE'))

    return available_slots


def poll_and_alert(client, alerter, poll_interval):
    while True:
        timeslots_html = client.get_delivery_timeslots_html()
        available_slots = parse_timeslots(timeslots_html)
        if available_slots:
            logger.info('***** TIMESLOTS ARE AVAILABLE ******')
            message = 'The following time slots are available on Fresh Direct:\n\n'
            for slot in available_slots:
                message += slot + '\n'
            logger.info(message)
            alerter.alert(message)
        time.sleep(poll_interval)
        
        
def run_main(poll_interval, alert_interval):
    """ Main entry point.
    
    Args
        poll_interval (int)
            Number of seconds to wait between refreshes.
        
        alert_interval (int)
            Number of seconds to wait between subsequent alerts.
    """
    start_time = datetime.datetime.now()
    
    client = FreshDirectClient(logger)
    client.authenticate(USERNAME, PASSWORD)
    
    alerter = TextAlerter(alert_interval)
    
    is_running = True
    while is_running:
        try:
            message = 'Fresh direct polling started.'
            alerter.alert(message)
            logger.info(message)
            poll_and_alert(client, alerter, poll_interval)
        except KeyboardInterrupt:
            logger.info('Stopped.')
            is_running = False
        except Exception as e:
            alerter.alert('fd-poller error: {}'.format(str(e)))
            stacktrace = ''.join(traceback.format_exception(*sys.exc_info()))
            logger.error(stacktrace)
        finally:
            end_time = datetime.datetime.now()
            run_time = end_time - start_time
            message = 'Fresh direct poller finished running.'
            alerter.alert(message)
            logger.info(message)

In [12]:
run_main(poll_interval=15, alert_interval=60)

2020-05-11 18:46:55 INFO     Authentication successful!
2020-05-11 18:46:56 ERROR    Traceback (most recent call last):
  File "<ipython-input-11-3ce21debc7e5>", line 124, in run_main
    alerter.alert(message)
  File "<ipython-input-11-3ce21debc7e5>", line 9, in alert
    self.user_alert(message)
  File "<ipython-input-11-3ce21debc7e5>", line 22, in user_alert
    {'reference': 'Foobar'}
  File "/Users/jameshadar/.virtualenvs/pydata-book/lib/python3.6/site-packages/messagebird/client.py", line 200, in message_create
    return Message().load(self.request('messages', 'POST', params))
  File "/Users/jameshadar/.virtualenvs/pydata-book/lib/python3.6/site-packages/messagebird/client.py", line 95, in request
    raise (ErrorException([Error().load(e) for e in response_json['errors']]))
messagebird.client.ErrorException: {'code': 25, 'description': 'Not enough balance', 'parameter': None}

2020-05-11 18:46:56 INFO     Fresh direct poller finished running.
2020-05-11 18:46:56 INFO     Fresh 